In [1]:
# export
from local.core.imports import *
from local.core.utils import compose
from local.test import *
from local.notebook.export import *
from local.notebook.showdoc import *
import nbformat
from notebook import notebookapp
from nbconvert.preprocessors import ExecutePreprocessor, Preprocessor
from nbconvert import HTMLExporter,MarkdownExporter
from nbformat.sign import NotebookNotary
from traitlets.config import Config

In [2]:
# default_exp notebook.export2html
# default_cls_lvl 3

# Converting notebooks to html

> The functions that transform the dev notebooks in the documentation of the library

## Preprocessing notebook

### Cell processors

In [3]:
#export
def remove_widget_state(cell):
    "Remove widgets in the output of `cells`"
    if cell['cell_type'] == 'code' and 'outputs' in cell:
        cell['outputs'] = [l for l in cell['outputs']
                           if not ('data' in l and 'application/vnd.jupyter.widget-view+json' in l.data)]
    return cell

In [4]:
#export
# Matches any cell that has a `show_doc` or an `#export` in it
_re_cell_to_hide = r's*show_doc\(|^\s*#\s*export\s+'

In [5]:
#export
def hide_cells(cell):
    "Hide `cell` that need to be hidden"
    if check_re(cell, _re_cell_to_hide):  cell['metadata'] = {'hide_input': True}
    return cell

In [6]:
for source in ['show_doc(read_nb)', '# export\nfrom local.core import *']:
    cell = {'cell_type': 'code', 'source': 'show_doc(read_nb)'}
    cell1 = hide_cells(cell.copy())
    assert 'metadata' in cell1
    assert 'hide_input' in cell1['metadata']
    assert cell1['metadata']['hide_input']

cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
test_eq(hide_cells(cell.copy()), cell)

In [7]:
#export
# Matches any line containing an #exports
_re_exports = re.compile(r'^#\s*exports[^\n]*\n')

In [8]:
#export
def clean_exports(cell):
    "Remove exports flag from `cell`"
    cell['source'] = _re_exports.sub('', cell['source'])
    return cell

In [9]:
cell = {'cell_type': 'code', 'source': '# exports\nfrom local.core import *'}
test_eq(clean_exports(cell.copy()), {'cell_type': 'code', 'source': 'from local.core import *'})
cell = {'cell_type': 'code', 'source': '# exports core\nfrom local.core import *'}
test_eq(clean_exports(cell.copy()), {'cell_type': 'code', 'source': 'from local.core import *'})

In [10]:
#export
def treat_backticks(cell):
    "Add links to backticks words in `cell`"
    if cell['cell_type'] == 'markdown': cell['source'] = add_doc_links(cell['source'])
    return cell

In [11]:
cell = {'cell_type': 'markdown', 'source': 'This is a `Tensor`'}
test_eq(treat_backticks(cell), {'cell_type': 'markdown',
    'source': 'This is a [`Tensor`](https://pytorch.org/docs/stable/tensors.html#torch-tensor)'})

In [12]:
#export
_re_nb_link = re.compile(r"""
# Catches any link to a local notebook and keeps the title in group 1, the link without .ipynb in group 2
\[          # Opening [
([^\]]*)    # Catching group for any character except ]
\]\(        # Closing ], opening (
([^http]    # Catching group that must not begin by html (local notebook)
[^\)]*)     # and containing anything but )
.ipynb\)    # .ipynb and closing )
""", re.VERBOSE)

In [13]:
# export
def convert_links(cell):
    "Convert the .ipynb links to .html"
    if cell['cell_type'] == 'markdown':
        cell['source'] = _re_nb_link.sub(r'[\1](\2.html)', cell['source'])
    return cell

In [14]:
cell = {'cell_type': 'markdown', 'source': "This is a link to a [notebook](01_core.ipynb)."}
test_eq(convert_links(cell), {'cell_type': 'markdown', 
                               'source': "This is a link to a [notebook](01_core.html)."})
cell = {'cell_type': 'markdown', 'source': "This is a link to a [page](01_core.html)."}
test_eq(convert_links(cell.copy()), cell)
cell = {'cell_type': 'markdown', 'source': "This is a link to an [external nb](http://01_core.ipynb)."}
test_eq(convert_links(cell.copy()), cell)

In [15]:
#export
_re_block_notes = re.compile(r"""
# Catches any pattern > Title: content with title in group 1 and content in group 2
^>\s*      # > followed by any number of whitespace
([^:]*)   # Catching group for any character but :
:\s*      # : then any number of whitespace
([^\n]*)  # Catching group for anything but a new line character
(?:\n|$)  # Non-catching group for either a new line or the end of the text
""", re.VERBOSE)

In [16]:
#export
def add_jekyll_notes(cell):
    "Convert block quotes to jekyll notes in `cell`"
    t2style = {'Note': 'info', 'Warning': 'danger', 'Important': 'warning'}
    def _inner(m):
        title,text = m.groups()
        style = t2style.get(title, None)
        if style is None: return f"> {m.groups()[0]}: {m.groups()[1]}"
        res = f'<div markdown="span" class="alert alert-{style}" role="alert">'
        return res + f'<i class="fa fa-{style}-circle"></i> <b>{title}: </b>{text}</div>'
    if cell['cell_type'] == 'markdown':
        cell['source'] = _re_block_notes.sub(_inner, cell['source'])
    return cell

Supported styles are `Warning`, `Note` and `Important`:

> Warning: There will be no second warning!

> Important: Pay attention! This is important.

> Note: Take note of this.

In [17]:
for w,s in zip(['Warning', 'Note', 'Important', 'Bla'], ['danger', 'info', 'warning', 'info']):
    cell = {'cell_type': 'markdown', 'source': f"> {w}: This is my final {w.lower()}!"}
    res = f'<div markdown="span" class="alert alert-{s}" role="alert">'
    res += f'<i class="fa fa-{s}-circle"></i> <b>{w}: </b>This is my final {w.lower()}!</div>'
    if w != 'Bla': test_eq(add_jekyll_notes(cell), {'cell_type': 'markdown', 'source': res})
    else: test_eq(add_jekyll_notes(cell), cell)

In [18]:
#export
_re_image = re.compile(r"""
# Catches any image file used, either with `![alt](image_file)` or `<img src="image_file">`
^!\[        #   Beginning of line (since re.MULTILINE is passed) followed by ![
[^\]]*      #   Anything but ]
\]\(        #   Closing ] and opening (
([^\)]*)    #   Catching block with any character but )
\)          #   Closing )
|           # OR
<img\ src="  #   <img src="
([^"]*)     #   Catching block with any character except "
"           #   Closing
""", re.MULTILINE | re.VERBOSE)

In [19]:
#export
def copy_images(cell, fname, dest):
    if cell['cell_type'] == 'markdown' and _re_image.search(cell['source']):
        grps = _re_image.search(cell['source']).groups()
        src = grps[0] or grps[1]
        os.makedirs((Path(dest)/src).parent, exist_ok=True)
        shutil.copy(Path(fname).parent/src, Path(dest)/src)
    return cell

In [20]:
dest_img = Path('docs')/'images'/'pixelshuffle.png' 
dest_bak = Path('docs')/'images'/'pixelshuffle.bak'
if dest_img.exists(): shutil.move(dest_img, dest_bak)
for text in ['Text\n![Alt](images/pixelshuffle.png)', 
             'Text\n<img src="images/pixelshuffle.png" alt="Pixelshuffle" style="width: 100%; height: auto;"/>']:
    cell = {'cell_type': 'markdown', 'source': text}
    cell1 = copy_images(cell, Path('10_layers.ipynb'), Path('docs'))
    #Function doesn't touch cell
    test_eq(cell, cell1)
    #Image has been copied
    assert dest_img.exists()
    os.remove(dest_img)
if dest_bak.exists(): shutil.move(dest_bak, dest_img)

In [21]:
#export
#Matches any cell with #hide or #default_exp or #default_cls_lvl
_re_cell_to_remove = re.compile(r'^\s*#\s*(hide|default_exp|default_cls_lvl)\s+')

In [22]:
#export
def remove_hidden(cells):
    "Remove in `cells` the ones with a flag `#hide` or `#default_exp`"
    return [c for c in cells if _re_cell_to_remove.search(c['source']) is None]

In [23]:
cells = [{'cell_type': 'code', 'source': source} for source in [
    '# export\nfrom local.core import *', 
    '# hide\nfrom local.core import *',
    '#exports\nsuper code',
    '#default_exp notebook.export',
    'show_doc(read_nb)',
    '#default_cls_lvl 3']] + [{'cell_type': 'markdown', 'source': source} for source in [
    'nice', '#hide\n\nto hide']]
         
cells1 = remove_hidden(cells)
test_eq(len(cells1), 4)
test_eq(cells1[0], cells[0])
test_eq(cells1[1], cells[2])
test_eq(cells1[2], cells[4])
test_eq(cells1[3], cells[6])

In [24]:
#export
_re_default_cls_lvl = re.compile(r"""
^               # Beginning of line (since re.MULTILINE is passed)
\s*\#\s*        # Any number of whitespace, #, any number of whitespace
default_cls_lvl # default_cls_lvl
\s*             # Any number of whitespace
(\d*)           # Catching group for any number of digits
\s*$            # Any number of whitespace and end of line (since re.MULTILINE is passed)
""", re.IGNORECASE | re.MULTILINE | re.VERBOSE)

In [25]:
# export
def find_default_level(cells):
    "Find in `cells` the default export module."
    for cell in cells:
        tst = check_re(cell, _re_default_cls_lvl)
        if tst: return int(tst.groups()[0])
    return 2

In [26]:
tst_nb = read_nb('91_notebook_export.ipynb')
test_eq(find_default_level(tst_nb['cells']), 3)

In [27]:
#export
#Find a cell with #export(s)
_re_export = re.compile(r'^\s*#\s*exports?\s*', re.IGNORECASE | re.MULTILINE)
_re_show_doc = re.compile(r"""
# First one catches any cell with a #export or #exports, second one catches any show_doc and get the first argument in group 1
show_doc     # show_doc
\s*\(\s*     # Any number of whitespace, opening (, any number of whitespace
([^,\)\s]*)  # Catching group for any character but a comma, a closing ) or a whitespace
[,\)\s]      # A comma, a closing ) or a whitespace
""", re.MULTILINE | re.VERBOSE)

In [28]:
#export
def _show_doc_cell(name, cls_lvl=None):
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {},
            'outputs': [],
            'source': f"show_doc({name}{'' if cls_lvl is None else f', default_cls_level={cls_lvl}'})"}

def add_show_docs(cells, cls_lvl=None):
    "Add `show_doc` for each exported function or class"
    documented = [_re_show_doc.search(cell['source']).groups()[0] for cell in cells
                  if cell['cell_type']=='code' and _re_show_doc.search(cell['source']) is not None]
    res = []
    for cell in cells:
        res.append(cell)
        if check_re(cell, _re_export):
            names = export_names(cell['source'], func_only=True)
            for n in names:
                if n not in documented: res.append(_show_doc_cell(n, cls_lvl=cls_lvl))
    return res

In [29]:
for i,cell in enumerate(tst_nb['cells']):
    if cell['source'].startswith('#export\ndef read_nb'): break
tst_cells = [c.copy() for c in tst_nb['cells'][i-1:i+1]]
added_cells = add_show_docs(tst_cells, cls_lvl=3)
test_eq(len(added_cells), 3)
test_eq(added_cells[0], tst_nb['cells'][i-1])
test_eq(added_cells[1], tst_nb['cells'][i])
test_eq(added_cells[2], _show_doc_cell('read_nb', cls_lvl=3))
test_eq(added_cells[2]['source'], 'show_doc(read_nb, default_cls_level=3)')

#Check show_doc isn't added if it was already there.
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'code', 'source': 'show_doc(my_func)'}]
test_eq(add_show_docs(tst_cells1), tst_cells1)
tst_cells1 = [{'cell_type':'code', 'source': '#export\ndef my_func(x):\n    return x'},
              {'cell_type':'markdown', 'source': 'Some text'},
              {'cell_type':'code', 'source': 'show_doc(my_func, title_level=3)'}]
test_eq(add_show_docs(tst_cells1), tst_cells1)

In [30]:
#export 
_re_fake_header = re.compile(r"""
# Matches any fake header (one that ends with -)
\#+    # One or more #
\s+    # One or more of whitespace
.*     # Any char
-\s*   # A dash followed by any number of white space
$      # End of text
""", re.VERBOSE)

In [31]:
# export
def remove_fake_headers(cells):
    "Remove in `cells` the fake header"
    return [c for c in cells if c['cell_type']=='code' or _re_fake_header.search(c['source']) is None]

In [32]:
cells = [{'cell_type': 'markdown',
          'metadata': {},
          'source': '### Fake-'}] + tst_nb['cells'][:10]
cells1 = remove_fake_headers(cells)
test_eq(len(cells1), len(cells)-1)
test_eq(cells1[0], cells[1])

In [33]:
# export
def remove_empty(cells):
    "Remove in `cells` the empty cells"
    return [c for c in cells if len(c['source']) >0]

### Grabbing metada

In [34]:
#export 
_re_title_summary = re.compile(r"""
# Catches the title and summary of the notebook, presented as # Title > summary, with title in group 1 and summary in group 2
^\s*       # Beginning of text followe by any number of whitespace
\#\s+      # # followed by one or more of whitespace
([^\n]*)   # Catching group for any character except a new line
\n+        # One or more new lines
>\s*       # > followed by any number of whitespace
([^\n]*)   # Catching group for any character except a new line
""", re.VERBOSE)

_re_properties = re.compile(r"""
^-\s+      # Beginnig of a line followed by - and at least one space
(.*?)      # Any pattern (shortest possible)
\s*:\s*    # Any number of whitespace, :, any number of whitespace
(.*?)$     # Any pattern (shortest possible) then end of line
""", re.MULTILINE | re.VERBOSE)

In [35]:
# export
def get_metadata(cells):
    "Find the cell with title and summary in `cells`."
    for i,cell in enumerate(cells):
        if cell['cell_type'] == 'markdown':
            match = _re_title_summary.match(cell['source'])
            if match:
                cells.pop(i)
                attrs = {k:v for k,v in _re_properties.findall(cell['source'])}
                return {'keywords': 'fastai',
                        'summary' : match.groups()[1],
                        'title'   : match.groups()[0],
                        **attrs}
    return {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'}

In [36]:
tst_nb = read_nb('91_notebook_export.ipynb')
test_eq(get_metadata(tst_nb['cells']), {
    'keywords': 'fastai',
    'summary': 'The functions that transform the dev notebooks in the fastai library',
    'title': 'Converting notebooks to modules',
    'author': '"Sylvain Gugger"'})
#The cell with the metada is poped out, so if we do it a second time we get the default.
test_eq(get_metadata(tst_nb['cells']), {'keywords': 'fastai',
            'summary' : 'summary',
            'title'   : 'Title'})

## Executing show_doc cells

In [37]:
#export
#Catches any cell with a show_doc or an import from local
_re_cell_to_execute = re.compile(r"^\s*show_doc\(([^\)]*)\)|^from local\.", re.MULTILINE)

In [38]:
# export
class ExecuteShowDocPreprocessor(ExecutePreprocessor):
    "An `ExecutePreprocessor` that only executes `show_doc` and `import` cells"
    def preprocess_cell(self, cell, resources, index):
        if 'source' in cell and cell['cell_type'] == "code":
            if _re_cell_to_execute.search(cell['source']):
                return super().preprocess_cell(cell, resources, index)
        return cell, resources

In [39]:
# export
def _import_show_doc_cell(mod=None, name=None):
    "Add an import show_doc cell + deal with the _file_ hack if necessary."
    source = f"#export\nfrom local.notebook.showdoc import show_doc"
    if mod:  source += f"\nfrom local.{mod} import *"
    if name: source += f"\nfrom pathlib import Path\n_file_ = {name}"
    return {'cell_type': 'code',
            'execution_count': None,
            'metadata': {'hide_input': True},
            'outputs': [],
            'source': source}

def execute_nb(nb, mod=None, metadata=None, show_doc_only=True, name=None):
    "Execute `nb` (or only the `show_doc` cells) with `metadata`"
    nb['cells'].insert(0, _import_show_doc_cell(mod, name))
    ep_cls = ExecuteShowDocPreprocessor if show_doc_only else ExecutePreprocessor
    ep = ep_cls(timeout=600, kernel_name='python3')
    metadata = metadata or {}
    pnb = nbformat.from_dict(nb)
    ep.preprocess(pnb, metadata)
    return pnb

In [40]:
fake_nb = {k:v for k,v in tst_nb.items() if k != 'cells'}
fake_nb['cells'] = [tst_nb['cells'][0].copy()] + added_cells
fake_nb = execute_nb(fake_nb, mod='notebook.export')
assert len(fake_nb['cells'][-1]['outputs']) > 0

## Conversion

In [41]:
#hide
#Tricking jupyter notebook to have a __file__ attribute. All _file_ will be replaced by __file__
_file_ = Path('local').absolute()/'notebook'/'export.py'

In [42]:
# export
def _exporter(markdown=False):
    cfg = Config()
    exporter = (HTMLExporter,MarkdownExporter)[markdown](cfg)
    exporter.exclude_input_prompt=True
    exporter.exclude_output_prompt=True
    exporter.template_file = ('jekyll.tpl','jekyll-md.tpl')[markdown]
    exporter.template_path.append(str(Path(_file_).parent))
    return exporter

In [43]:
# export
process_cells = [remove_fake_headers, remove_hidden, remove_empty]
process_cell  = [hide_cells, remove_widget_state, add_jekyll_notes, convert_links]

In [44]:
#export
_re_file = re.compile(r"""
^_file_   # _file_ at the beginning of a line (since re.MULTILINE is passed)
\s*=\s*   # Any number of whitespace, =, any number of whitespace
(\S*)     # Catching group for any non-whitespace characters
\s*$      # Any number of whitespace then the end of line
""", re.MULTILINE | re.VERBOSE)

In [45]:
# export
def _find_file(cells):
    "Find in `cells` if a _file_ is defined."
    for cell in cells:
        if cell['cell_type']=='code' and _re_file.search(cell['source']):
            return _re_file.search(cell['source']).groups()[0]

In [46]:
#hide
tst_nb = read_nb('91_notebook_export.ipynb')
test_eq(_find_file(tst_nb['cells']), "Path('local').absolute()/'notebook'/'export.py'")

In [47]:
#export
def notebook_path():
    "Returns the absolute path of the Notebook or None if it cannot be determined"
    #NOTE: works only when the security is token-based or there is no password
    kernel_id = Path(ipykernel.get_connection_file()).stem.split('-', 1)[1]
    for srv in notebookapp.list_running_servers():
        try:
            sessions = json.load(urlopen(f"{srv['url']}api/sessions{srv['token']}"))
            return next(Path(srv['notebook_dir'])/sess['notebook']['path']
                        for sess in sessions if sess['kernel']['id']==kernel_id)
        except: pass  # There may be stale entries in the runtime directory

In [48]:
#test_eq(notebook_path().name, '93_notebook_export2html.ipynb')
#test_eq(notebook_path().parent, Path().absolute())

In [49]:
# export
def convert_nb(fname, dest_path='docs'):
    "Convert a notebook `fname` to html file in `dest_path`."
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    cls_lvl = find_default_level(nb['cells'])
    _name = _find_file(nb['cells'])
    mod = find_default_export(nb['cells'])
    nb['cells'] = compose(*process_cells,partial(add_show_docs, cls_lvl=cls_lvl))(nb['cells'])
    nb['cells'] = [compose(partial(copy_images, fname=fname, dest=dest_path), *process_cell, treat_backticks)(c)
                    for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = '.'.join(fname.with_suffix('.html').name.split('_')[1:])
    meta_jekyll = get_metadata(nb['cells'])
    meta_jekyll['nb_path'] = f'{fname.parent.name}/{fname.name}'
    nb = execute_nb(nb, mod=mod, name=_name)
    nb['cells'] = [clean_exports(c) for c in nb['cells']]
    #print(f'{dest_path}/{dest_name}')
    with open(f'{dest_path}/{dest_name}','w') as f:
        #res = _exporter().from_notebook_node(nb, resources=meta_jekyll)[0]
        #print(res)
        f.write(_exporter().from_notebook_node(nb, resources=meta_jekyll)[0])

In [50]:
convert_nb('91_notebook_export.ipynb', '../docs')

In [51]:
# export
def convert_all(path='.', dest_path='../docs', force_all=False):
    "Convert all notebooks in `path` to html files in `dest_path`."
    path = Path(path)
    changed_cnt = 0
    for fname in path.glob("[0-9]*.ipynb"):
        # only rebuild modified files
        if fname.name.startswith('_'): continue
        fname_out = Path(dest_path)/'.'.join(fname.with_suffix('.html').name.split('_')[1:])
        if not force_all and fname_out.exists() and os.path.getmtime(fname) < os.path.getmtime(fname_out):
            continue
        print(f"converting: {fname} => {fname_out}")
        changed_cnt += 1
        try: convert_nb(fname, dest_path=dest_path)
        except Exception as e: print(e)
    if changed_cnt==0: print("No notebooks were modified")

In [ ]:
#hide
#convert_all(force_all=True)

In [53]:
# export
def convert_post(fname, dest_path='posts'):
    "Convert a notebook `fname` to blog post markdown in `dest_path`."
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    meta_jekyll = get_metadata(nb['cells'])
    nb['cells'] = compose(*process_cells)(nb['cells'])
    nb['cells'] = [compose(*process_cell)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    dest_name = fname.with_suffix('.md').name
    exp = _exporter(markdown=True)
    with (Path(dest_path)/dest_name).open('w') as f:
        f.write(exp.from_notebook_node(nb, resources=meta_jekyll)[0])

In [ ]:
convert_post('posts/2019-08-06-delegation.ipynb')

## Export-

In [54]:
#hide
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_core_foundation.ipynb.
Converted 01a_core_utils.ipynb.
Converted 01b_core_dispatch.ipynb.
Converted 01c_core_transform.ipynb.
Converted 02_core_script.ipynb.
Converted 03_torchcore.ipynb.
Converted 03a_layers.ipynb.
Converted 04_data_load.ipynb.
Converted 05_data_core.ipynb.
Converted 06_data_transforms.ipynb.
Converted 07_data_block.ipynb.
Converted 08_vision_core.ipynb.
Converted 09_vision_augment.ipynb.
Converted 09a_vision_data.ipynb.
Converted 10_pets_tutorial.ipynb.
Converted 11_vision_models_xresnet.ipynb.
Converted 12_optimizer.ipynb.
Converted 13_learner.ipynb.
Converted 13a_metrics.ipynb.
Converted 14_callback_schedule.ipynb.
Converted 14a_callback_data.ipynb.
Converted 15_callback_hook.ipynb.
Converted 15a_vision_models_unet.ipynb.
Converted 16_callback_progress.ipynb.
Converted 17_callback_tracker.ipynb.
Converted 18_callback_fp16.ipynb.
Converted 19_callback_mixup.ipynb.
Converted 20_interpret.ipynb.
Converted 20a_distributed.ipynb.
Co

In [ ]:
#hide
def debug_nb(fname, dest=None):
    fname = Path(fname).absolute()
    nb = read_nb(fname)
    cls_lvl = find_default_level(nb['cells'])
    _name = _find_file(nb['cells'])
    nb['cells'] = compose(*process_cells, partial(add_show_docs, cls_lvl=cls_lvl))(nb['cells'])
    nb['cells'] = [compose(*process_cell)(c) for c in nb['cells']]
    fname = Path(fname).absolute()
    nb = execute_nb(nb, name=_name)
    dest = dest or fname.with_suffix('.dbg.ipynb')
    nbformat.write(nbformat.from_dict(nb), open(dest, 'w'), version=4)

In [ ]:
#hide
# debug_nb('93_notebook_export2html.ipynb')